# Homework 3

## Pair Programming Group Number: FILL IN HERE
## Members of Team: FILL IN HERE

## Feature engineering and linear regression

For this week's homework we are going to load in a data set that isn't in the "cleanest", repair it, add a feature, do some analysis on the features, build a linear regression model, and use that model to estimate numeric values.  Is linear regression _really_ machine learning? Depends on who you ask, but it is definitely an important tool for data analytics. 

In [1]:
# only use these libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load in the melb_data_sold_train.csv file here
df = pd.read_csv('melb_data_sold_train.csv')

## Q1 Fix the dataframe to remove any blanks
The linear regression needs all attribute and dependent values to be defined.  Use list-wise deletion to remove entries with missing values. Save the modified dataframe with the indices reset to be $0-(length-1)$ into the variable `df1` for use in a later problem. 

In [2]:
df1 = df.dropna().reset_index()

## Q2 Add a new feature
Toorak is known as one of the priciest suburbs in Melbourne.  Create a new column in your dataframe that is the distance in kilometers from the center of Toorak to the latitude/longitude of that row.  Use the latitude / longitude of $(-37.841820, 145.015986)$ for the center of Toorak.  You may assume the Earth is spherical and has radius of $6371.0088$km (check your function ... the property located at $(-37.68178,144.73779)$ is approx 30.2 km away)

In [3]:
# Step 1 : Define the Haversine distance as a function
# assumes that pt1 and pt2 are 2x1 [lat,long] np arrays that contain locations of the 2 earth coordinates in deg
# using the Haversine formula found https://en.wikipedia.org/wiki/Haversine_formula
def haversine_distance(pt1,pt2):
    lat1 = pt1[0]*np.pi/180
    lat2 = pt2[0]*np.pi/180
    lon1 = pt1[1]*np.pi/180
    lon2 = pt2[1]*np.pi/180
    havTheta = hav(lat2-lat1)+np.cos(lat1)*np.cos(lat2)*hav(lon2-lon1)
    d_r = np.arccos(1-havTheta*2)
    return d_r *6371.0088

#code here, make sure pt1 and pt2 are passed in as degrees (lat,long) and convert to radians before calculation

def hav(theta):
    return (np.sin(theta/2))**2

In [4]:
# A quick check to see if we are getting the expected value
toorak_pt = np.array([-37.841820, 145.015986])
haversine_distance(toorak_pt,[-37.68178,144.73779])


30.244772513599806

In [5]:
# Step 2: Add a new column to `df1` called 'distance_to_toorak' that uses the haversine_distance function 
# to calculate the distance to Toorak for every row in our dataframe. Save the new dataframe as `df2`
# toorak_pt = np.array([-37.841820, 145.015986])
df2 = df1.assign(distance_to_toorak = haversine_distance(toorak_pt, np.array([df1["Lattitude"], df1["Longtitude"]])))

df2

,index,Suburb,Address,Rooms,Type,Price,Date,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,distance_to_toorak
0,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,4/02/2016,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4.261612
1,3,Abbotsford,124 Yarra St,3,h,1876000.0,7/05/2016,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.80240,144.99930,Northern Metropolitan,4.621843
2,4,Abbotsford,98 Charles St,2,h,1636000.0,8/10/2016,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.80600,144.99540,Northern Metropolitan,4.374206
3,6,Abbotsford,10 Valiant St,2,h,1097000.0,8/10/2016,3067.0,3.0,1.0,2.0,220.0,75.0,1900.0,Yarra,-37.80100,144.99890,Northern Metropolitan,4.780655
4,7,Abbotsford,123/56 Nicholson St,2,u,750000.0,12/11/2016,3067.0,2.0,2.0,1.0,0.0,94.0,2009.0,Yarra,-37.80780,144.99650,Northern Metropolitan,4.152012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,7985,Glenroy,69 Melbourne Av,3,h,680000.0,29/07/2017,3046.0,3.0,1.0,1.0,422.0,104.0,1960.0,Moreland,-37.70425,144.93164,Northern Metropolitan,16.998846
3906,7988,Greensborough,12 Yangoora Pl,4,h,900000.0,29/07/2017,3088.0,4.0,3.0,1.0,807.0,190.0,1975.0,Banyule,-37.70077,145.12743,Northern Metropolitan,18.491623
3907,7995,Highett,1/12 Jillian Av,3,t,1200000.0,29/07/2017,3190.0,3.0,1.0,2.0,269.0,125.0,2000.0,Bayside,-37.94775,145.02379,Southern Metropolitan,11.798784
3908,7997,Highett,31 The Crescent,4,t,1200000.0,29/07/2017,3190.0,4.0,2.0,2.0,304.0,183.0,2007.0,Kingston,-37.94633,145.04947,Southern Metropolitan,11.986684


## Q3 Create a one hot encoding for the categorical column 'Type'
Modify the data frame `df2` such that it removes the column for `Type` and replaces it with the appropriate number of columns for a one-hot encoding of the column `Type` and save that dataframe as `df3` for use in a later problem. The pandas method [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) will be very useful here. 

In [21]:
df3 = df2.drop("Type",1)
ind = df2.columns.get_loc("Type")

ohe = pd.get_dummies(df2["Type"],columns="Type")


for i in range(len(ohe.columns)):
    df3.insert(ind+i,ohe.columns[i],ohe[ohe.columns[i]])
    
df3

,index,Suburb,Address,Rooms,h,t,u,Price,Date,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,distance_to_toorak
0,1,Abbotsford,25 Bloomburg St,2,1,0,0,1035000.0,4/02/2016,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4.261612
1,3,Abbotsford,124 Yarra St,3,1,0,0,1876000.0,7/05/2016,3067.0,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.80240,144.99930,Northern Metropolitan,4.621843
2,4,Abbotsford,98 Charles St,2,1,0,0,1636000.0,8/10/2016,3067.0,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.80600,144.99540,Northern Metropolitan,4.374206
3,6,Abbotsford,10 Valiant St,2,1,0,0,1097000.0,8/10/2016,3067.0,...,1.0,2.0,220.0,75.0,1900.0,Yarra,-37.80100,144.99890,Northern Metropolitan,4.780655
4,7,Abbotsford,123/56 Nicholson St,2,0,0,1,750000.0,12/11/2016,3067.0,...,2.0,1.0,0.0,94.0,2009.0,Yarra,-37.80780,144.99650,Northern Metropolitan,4.152012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,7985,Glenroy,69 Melbourne Av,3,1,0,0,680000.0,29/07/2017,3046.0,...,1.0,1.0,422.0,104.0,1960.0,Moreland,-37.70425,144.93164,Northern Metropolitan,16.998846
3906,7988,Greensborough,12 Yangoora Pl,4,1,0,0,900000.0,29/07/2017,3088.0,...,3.0,1.0,807.0,190.0,1975.0,Banyule,-37.70077,145.12743,Northern Metropolitan,18.491623
3907,7995,Highett,1/12 Jillian Av,3,0,1,0,1200000.0,29/07/2017,3190.0,...,1.0,2.0,269.0,125.0,2000.0,Bayside,-37.94775,145.02379,Southern Metropolitan,11.798784
3908,7997,Highett,31 The Crescent,4,0,1,0,1200000.0,29/07/2017,3190.0,...,2.0,2.0,304.0,183.0,2007.0,Kingston,-37.94633,145.04947,Southern Metropolitan,11.986684


## Q4 Calculate the pairwise correlations between all of your numeric attributes
Use the Pearson correlation as discussed in the lectures to calculate the pairwise correlations between the attributes in the dataframe `df3`. Read the documentation for [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html). 

In [7]:
pCorr = df3.corr(method="pearson")
pCorr

,index,Rooms,Type_h,Type_t,Type_u,Price,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,distance_to_toorak
index,1.000000,0.041668,0.016705,0.009308,-0.025499,-0.156108,0.063008,0.064716,0.013282,0.077934,0.073302,-0.021849,0.155483,0.086635,0.035259,0.359392
Rooms,0.041668,1.000000,0.501128,-0.040762,-0.535102,0.502814,0.054676,0.941684,0.593847,0.401716,0.085676,0.547816,-0.030447,0.012858,0.074846,0.232844
Type_h,0.016705,0.501128,1.000000,-0.484749,-0.778062,0.391761,-0.040504,0.484612,0.181773,0.214175,0.047422,0.306231,-0.402670,0.121254,-0.040387,0.221104
Type_t,0.009308,-0.040762,-0.484749,1.000000,-0.172282,-0.095295,-0.008546,-0.037366,0.108148,-0.006836,-0.067257,-0.032396,0.331105,-0.028636,0.023603,-0.014192
Type_u,-0.025499,-0.535102,-0.778062,-0.172282,1.000000,-0.372766,0.051754,-0.518941,-0.282390,-0.236298,-0.005103,-0.321614,0.215689,-0.115991,0.028533,-0.238818
Price,-0.156108,0.502814,0.391761,-0.095295,-0.372766,1.000000,0.136468,0.480115,0.462985,0.223552,0.064029,0.465598,-0.330560,-0.224400,0.184597,-0.357013
Postcode,0.063008,0.054676,-0.040504,-0.008546,0.051754,0.136468,1.000000,0.059311,0.122046,0.034698,0.014276,0.089355,0.035561,-0.445625,0.471934,0.008062
Bedroom2,0.064716,0.941684,0.484612,-0.037366,-0.518941,0.480115,0.059311,1.000000,0.594920,0.403840,0.083237,0.532910,-0.010031,0.019970,0.076236,0.241515
Bathroom,0.013282,0.593847,0.181773,0.108148,-0.282390,0.462985,0.122046,0.594920,1.000000,0.322266,0.059483,0.470554,0.192470,-0.068643,0.109313,0.051662
Car,0.077934,0.401716,0.214175,-0.006836,-0.236298,0.223552,0.034698,0.403840,0.322266,1.000000,0.100686,0.299613,0.153313,-0.007839,0.032915,0.214759


## Q5 Create a linear regression model to predict home values
Using the math in ESLII, section 3.2 equation (3.6) calculate $\hat{\beta}$

We are going to create a linear regression model using our numeric attribute columns in `df3`, and specifying the home values (`Price` column) as the value we are trying to predict.  You may use numpy to do matrix calculations, but you may not use a built in regression library (for example, you may not use scikt-learn). 

The features you use to build the matrix $X$ should all be numeric and include the distance to Toorak and the one hot encodings. 

In [22]:
# Step one, build the matrix X
numericCols = df3.select_dtypes(include=[np.number])
numericCols.insert(0,"intercept",1)
X = numericCols.drop("Price",1).drop("index",1).drop("Postcode",1)#.to_numpy()
#X.shape
print (X)

      intercept  Rooms  h  t  u  Bedroom2  Bathroom  Car  Landsize  \
0             1      2  1  0  0       2.0       1.0  0.0     156.0   
1             1      3  1  0  0       4.0       2.0  0.0     245.0   
2             1      2  1  0  0       2.0       1.0  2.0     256.0   
3             1      2  1  0  0       3.0       1.0  2.0     220.0   
4             1      2  0  0  1       2.0       2.0  1.0       0.0   
...         ...    ... .. .. ..       ...       ...  ...       ...   
3905          1      3  1  0  0       3.0       1.0  1.0     422.0   
3906          1      4  1  0  0       4.0       3.0  1.0     807.0   
3907          1      3  0  1  0       3.0       1.0  2.0     269.0   
3908          1      4  0  1  0       4.0       2.0  2.0     304.0   
3909          1      4  1  0  0       4.0       2.0  2.0     542.0   

      BuildingArea  YearBuilt  Lattitude  Longtitude  distance_to_toorak  
0             79.0     1900.0  -37.80790   144.99340            4.261612  
1        

In [23]:
# Step two, build the column vector y
y = numericCols["Price"].to_numpy()
#y.shape
y = y.reshape(y.shape[0],1)
#y

In [24]:
# Step three, find beta hat per the formula (3.6) (you should use the library we used in class)
beta_hat = np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(X.to_numpy()),X.to_numpy())),np.transpose(X.to_numpy())),y)
beta_hat

array([[ 2.71846740e+08],
       [-7.06477813e+05],
       [-2.94259824e+08],
       [-2.95368224e+08],
       [-2.97811166e+08],
       [ 2.65191413e+04],
       [ 2.11444638e+05],
       [ 5.27916046e+04],
       [ 1.96391910e+01],
       [ 1.24398796e+03],
       [-3.20399602e+03],
       [-8.53091862e+05],
       [ 1.25436059e+03],
       [-4.15728653e+04]])

In [25]:
# Test the model to see if we get something "reasonable" - i picked 23 at random 
np.matmul(X.iloc[23],beta_hat) 

array([1306651.53705806])

In [12]:
# This is the actual price at this point
y[23]

array([805000.])

## Q6 Apply the linear regression model to the test data and visualize the error
We will cover other methods of evaluating any sort of prediction later, but for this week's exercise I have partitioned the data into two files.  Load the melb_data_sold_test.csv data set and use the $\hat{\beta}$ you calculated in the last step to predict the housing prices for data in melb_data_sold_test.  Create a visualization that shows the absolute error in your predictions. Remember to do all your data pre-processing on the data loaded from the melb_data_sold_test file before you apply beta_hat.  For the visualization, a histogram of the absolute error vs the total housing prices is sufficient.  Use [hist](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.hist.html) for reference. 

While doing imputation, there are some helpful parameters in [fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html). 

In [13]:
# Step zero, load the melb_data_sold_test.csv data for testing.  Use Imputation to fill in any missing numeric values
# We use imputation here instead of deletion since we want a prediction for _every_ row in the test file.

In [14]:
# Step one, add the new attribute for the 'distance_to_toorak' and the one hot encoding to the new data frame

In [15]:
# Step two, build the attribute matrix Xdot 

In [16]:
# Step three, multiply Xdot by Beta hat. DO NOT USE A LOOP.  This is a vector of predicted prices
# called y_hat in the notes

In [17]:
# Step four, calculate the error vector, |actual price - predicted price|. We call this our "absolute error"

In [18]:
# Step five, create a histogram of the absolute error, and on the same plot create a histogram of the actual price.  
# You should use the "alpha" parameter to make the graph on top slightly translucent 